In [1]:
!pip install evaluate
!pip install jiwer
!pip install tqdm

In [8]:
!ps -ef | grep train_lora_script


root        3684    2435  0 10:15 ?        00:00:00 /bin/bash -c ps -ef | grep train_lora_script
root        3686    3684  0 10:15 ?        00:00:00 grep train_lora_script


In [5]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!ls -R /content/drive/MyDrive/project/whisper_lora_results


/content/drive/MyDrive/project/whisper_lora_results:
checkpoint-125	checkpoint-2201  checkpoint-4402  checkpoint-6603

/content/drive/MyDrive/project/whisper_lora_results/checkpoint-125:
adapter_config.json	   preprocessor_config.json  tokenizer_config.json
adapter_model.safetensors  README.md		     tokenizer.json
added_tokens.json	   rng_state.pth	     trainer_state.json
merges.txt		   scaler.pt		     training_args.bin
normalizer.json		   scheduler.pt		     vocab.json
optimizer.pt		   special_tokens_map.json

/content/drive/MyDrive/project/whisper_lora_results/checkpoint-2201:
adapter_config.json	   preprocessor_config.json  tokenizer_config.json
adapter_model.safetensors  README.md		     tokenizer.json
added_tokens.json	   rng_state.pth	     trainer_state.json
merges.txt		   scaler.pt		     training_args.bin
normalizer.json		   scheduler.pt		     vocab.json
optimizer.pt		   special_tokens_map.json

/content/drive/MyDrive/project/whisper_lora_results/checkpoint-4402:
adapter_config.js

In [ ]:
import os
import json
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio
from dataclasses import dataclass
from typing import Any

from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
)
import evaluate

# ============================================================
# 1. Configuration
# ============================================================
#DATA_ROOT = Path(r"/content/drive/MyDrive/cadenza_data")
DATA_ROOT = Path("/content/drive/MyDrive")
TRAIN_AUDIO_DIR = DATA_ROOT / "train" / "signals"
VALID_AUDIO_DIR = DATA_ROOT / "valid" / "signals"
TRAIN_METADATA_PATH = DATA_ROOT / "metadata" / "train_metadata.json"
VALID_METADATA_PATH = DATA_ROOT / "metadata" / "valid_metadata.json"  # optional

MODEL_NAME = "openai/whisper-small"  # or "openai/whisper-large-v2"
LANGUAGE = "English"
TASK = "transcribe"

EXPERIMENT_ID = "exp4"
EXPERIMENT_NAME = f"whisper_lora_{EXPERIMENT_ID}"

OUTPUT_DIR = f"/content/drive/MyDrive/project/{EXPERIMENT_NAME}_results"
SAVE_DIR = f"/content/drive/MyDrive/project/{EXPERIMENT_NAME}_finetuned"

print(f"📂 Experiment: {EXPERIMENT_NAME}")
print(f"📁 Training results will be saved to: {OUTPUT_DIR}")
print(f"📁 Final LoRA model will be saved to: {SAVE_DIR}")

BATCH_SIZE = 4
NUM_EPOCHS = 3
LEARNING_RATE = 1e-3
TARGET_SR = 16000

device = "cuda" if torch.cuda.is_available() else "cpu"

# Print device here
print("🔍 CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("🧠 GPU device:", torch.cuda.get_device_name(0))
    print("🔥 Current device index:", torch.cuda.current_device())
else:
    print("⚠️ Using CPU — training will be very slow")


# ============================================================
# 2. Dataset Definition
# ============================================================
class AudioPromptDataset(Dataset):
    def __init__(self, metadata_path, audio_dir, processor, target_sr=16000):
        with open(metadata_path, "r") as f:
            data = json.load(f)
        # handle nested list like [[{...}]]
        self.metadata = data[0] if isinstance(data, list) and isinstance(data[0], list) else data
        self.audio_dir = Path(audio_dir)
        self.processor = processor
        self.target_sr = target_sr

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        entry = self.metadata[idx]
        signal_id = entry["signal"]
        prompt = entry.get("prompt") or entry.get("original_prompt", "")

        audio_path = self.audio_dir / f"{signal_id}.flac"
        waveform, sr = torchaudio.load(audio_path)

        # convert to mono if stereo
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # resample to target_sr
        if sr != self.target_sr:
            waveform = torchaudio.functional.resample(waveform, sr, self.target_sr)

        # extract input features
        input_features = self.processor.feature_extractor(
            waveform.squeeze().numpy(), sampling_rate=self.target_sr
        ).input_features[0]

        # tokenize the text
        labels = self.processor.tokenizer(prompt, return_tensors="pt").input_ids.squeeze(0)

        return {"input_features": input_features, "labels": labels}


# ============================================================
# 3. Collator Function
# ============================================================
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features):
        input_features = [{"input_features": f["input_features"]} for f in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": f["labels"]} for f in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # remove duplicate BOS if exists
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch


# ============================================================
# 4. Load Processor and Dataset
# ============================================================
feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_NAME)
tokenizer = WhisperTokenizer.from_pretrained(MODEL_NAME, language=LANGUAGE, task=TASK)
processor = WhisperProcessor.from_pretrained(MODEL_NAME, language=LANGUAGE, task=TASK)

train_dataset = AudioPromptDataset(TRAIN_METADATA_PATH, TRAIN_AUDIO_DIR, processor, TARGET_SR)
valid_dataset = (
    AudioPromptDataset(VALID_METADATA_PATH, VALID_AUDIO_DIR, processor, TARGET_SR)
    if VALID_METADATA_PATH.exists()
    else None
)
collator = DataCollatorSpeechSeq2SeqWithPadding(processor)

# ============================================================
# 4.5. Quick Subset Training Mode (for fast loss verification)
# ============================================================

# ✅ Toggle mode: True = small subset for quick validation / False = full dataset for final training
USE_SMALL_SUBSET = False    # ⬅️ Change to False for full training
SUBSET_SIZE = 500           # Number of samples to use in subset
EPOCHS_SMALL = 1            # Use fewer epochs for quick test

if USE_SMALL_SUBSET:
    from torch.utils.data import Subset
    import random

    random.seed(42)
    indices = random.sample(range(len(train_dataset)), min(SUBSET_SIZE, len(train_dataset)))
    train_dataset = Subset(train_dataset, indices)

    print(f"🧪 Using a small subset of {len(indices)} samples for quick loss check.")
    NUM_EPOCHS = EPOCHS_SMALL  # Temporarily set fewer epochs for subset training
else:
    print("🏋️‍♀️ Using the full training dataset for final training.")


# ============================================================
# 5. Load Whisper Model (16-bit) and Prepare for LoRA
# ============================================================
model = WhisperForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    # torch_dtype=torch.float8_e4m3fn if torch.cuda.is_available() else torch.float8_e4m3fnuz,
    dtype=torch.float16,
    device_map="auto"
)

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

model = prepare_model_for_kbit_training(model)

# LoRA setup
# lora_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     target_modules=["q_proj", "v_proj"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type="SEQ_2_SEQ_LM",
# )


lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules = ["q_proj", "v_proj", "fc1", "fc2"],
    lora_dropout=0.1,
    bias="lora_only",
    task_type="SEQ_2_SEQ_LM",
)
model = get_peft_model(model, lora_config)
print("Trainable parameters:")
model.print_trainable_parameters()
# ============================================================
# 6. Evaluation Metric (WER)
# ============================================================
wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}


# ============================================================
# 7. Trainer Setup
# ============================================================
training_args = Seq2SeqTrainingArguments(
    #output_dir="./whisper_lora_results",
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=1,
    learning_rate=LEARNING_RATE,
    warmup_steps=50,
    num_train_epochs=NUM_EPOCHS,
    eval_strategy="no",
    # eval_strategy="epoch" if valid_dataset else "no",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=1,
    logging_dir="./logs",
    fp16=True,
    remove_unused_columns=False,
    label_names=["labels"],
    report_to="none",
    per_device_eval_batch_size=4,
    # eval_accumulation_steps=1,
    # predict_with_generate=False
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset if valid_dataset else None,
    data_collator=collator,
    tokenizer=processor,
    compute_metrics=compute_metrics if valid_dataset else None,
)

model.config.use_cache = False  # disable for training

# ============================================================
# 8. Train and Save
# ============================================================
from types import MethodType

def safe_forward(self, *args, **kwargs):
    # Drop 'input_ids' if it appears (Trainer sometimes injects it)
    kwargs.pop("input_ids", None)
    return self.base_model.forward(*args, **kwargs)

# Attach to PEFT model
model.forward = MethodType(safe_forward, model)

trainer.train()

#SAVE_DIR = "./whisper_lora_finetuned"
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR)
processor.save_pretrained(SAVE_DIR)
print(f"✅ Training complete! Model saved to: {SAVE_DIR}")

📂 Experiment: whisper_lora_exp4
📁 Training results will be saved to: /content/drive/MyDrive/project/whisper_lora_exp4_results
📁 Final LoRA model will be saved to: /content/drive/MyDrive/project/whisper_lora_exp4_finetuned
🔍 CUDA available: True
🧠 GPU device: Tesla T4
🔥 Current device index: 0
🏋️‍♀️ Using the full training dataset for final training.
Trainable parameters:
trainable params: 9,584,640 || all params: 251,172,096 || trainable%: 3.8160


/tmp/ipython-input-1610030668.py:250: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecat

Step,Training Loss
1,8.002300
2,8.377600
3,8.242400
4,8.580600
5,6.819000
6,8.064700
7,6.524000
8,6.169600
9,5.515800
10,5.230200


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

✅ Training complete! Model saved to: /content/drive/MyDrive/project/whisper_lora_exp4_finetuned


In [11]:
!ls /content/drive/MyDrive/project/whisper_lora_finetuned

adapter_config.json	   normalizer.json	     tokenizer_config.json
adapter_model.safetensors  preprocessor_config.json  tokenizer.json
added_tokens.json	   README.md		     vocab.json
merges.txt		   special_tokens_map.json


In [14]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel
import numpy as np
import torch
import torchaudio
from pathlib import Path
import os
import json



class AudioPromptDataset(torch.utils.data.Dataset):
    def __init__(self, metadata_path, audio_dir, processor, target_sr=16000):
        with open(metadata_path, "r") as f:
            data = json.load(f)
        # handle nested list like [[{...}]]
        self.metadata = data[0] if isinstance(data, list) and isinstance(data[0], list) else data
        self.audio_dir = Path(audio_dir)
        self.processor = processor
        self.target_sr = target_sr

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        entry = self.metadata[idx]
        signal_id = entry["signal"]
        prompt = entry.get("prompt") or entry.get("original_prompt", "")

        audio_path = self.audio_dir / f"{signal_id}.flac"
        waveform, sr = torchaudio.load(audio_path)

        # convert to mono if stereo
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # resample
        if sr != self.target_sr:
            waveform = torchaudio.functional.resample(waveform, sr, self.target_sr)

        # extract input features
        input_features = self.processor.feature_extractor(
            waveform.squeeze().numpy(), sampling_rate=self.target_sr
        ).input_features[0]

        # tokenize text
        labels = self.processor.tokenizer(prompt, return_tensors="pt").input_ids.squeeze(0)

        return {"input_features": input_features, "labels": labels, "metadata": entry}

DATA_ROOT = Path("/content/drive/MyDrive")
TRAIN_AUDIO_DIR = DATA_ROOT / "train" / "signals"
VALID_AUDIO_DIR = DATA_ROOT / "valid" / "signals"
TRAIN_METADATA_PATH = DATA_ROOT / "metadata" / "train_metadata.json"
VALID_METADATA_PATH = DATA_ROOT / "metadata" / "valid_metadata.json"  # optional

MODEL_NAME = "openai/whisper-small"  # or "openai/whisper-large-v2"
LANGUAGE = "English"
TASK = "transcribe"

EXPERIMENT_ID = "exp4"
EXPERIMENT_NAME = f"whisper_lora_{EXPERIMENT_ID}"

OUTPUT_DIR = f"/content/drive/MyDrive/project/{EXPERIMENT_NAME}_results"
SAVE_DIR = f"/content/drive/MyDrive/project/{EXPERIMENT_NAME}_finetuned"

# ===== Re-initialize core settings =====
DATA_ROOT = Path("/content/drive/MyDrive")

MODEL_NAME = "openai/whisper-small"
EXPERIMENT_ID = "exp4"
EXPERIMENT_NAME = f"whisper_lora_{EXPERIMENT_ID}"

OUTPUT_DIR = f"/content/drive/MyDrive/project/{EXPERIMENT_NAME}_results"
SAVE_DIR = f"/content/drive/MyDrive/project/{EXPERIMENT_NAME}_finetuned"

TARGET_SR = 16000
LANGUAGE = "English"
TASK = "transcribe"
device = "cuda" if torch.cuda.is_available() else "cpu"

# ===== Load processor (needed for feature extraction & decoding) =====
processor = WhisperProcessor.from_pretrained(SAVE_DIR)
print("✅ Processor loaded successfully!")


TEST_AUDIO_DIR = DATA_ROOT / "valid" / "signals"
TEST_METADATA_PATH = DATA_ROOT / "metadata" / "valid_metadata.json"

test_dataset = AudioPromptDataset(TEST_METADATA_PATH, TEST_AUDIO_DIR, processor, TARGET_SR)

# --- load saved adapter on top of base ---
SAVE_DIR = "/content/drive/MyDrive/project/whisper_lora_finetuned"
base_model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
model = PeftModel.from_pretrained(base_model, SAVE_DIR)  # LoRA adapters
processor = WhisperProcessor.from_pretrained(SAVE_DIR)

model.to(device)
model.eval()

# (Optional) If you want language/task tokens automatically:
# ids = processor.get_decoder_prompt_ids(language=LANGUAGE.lower(), task=TASK)
# model.generation_config.forced_decoder_ids = ids  # or leave None if you trained that way

def transcribe_sample(audio_path, max_new_tokens=128):
    waveform, sr = torchaudio.load(audio_path)

    # mono
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # resample
    if sr != TARGET_SR:
        waveform = torchaudio.functional.resample(waveform, sr, TARGET_SR)

    # extract features -> numpy array [80, T]
    feats = processor.feature_extractor(
        waveform.squeeze().numpy(), sampling_rate=TARGET_SR
    ).input_features[0]  # np.ndarray

    # convert to torch tensor with batch dim and move to device
    input_features = torch.from_numpy(np.array(feats)).unsqueeze(0).to(device)

    # fp16 autocast helps on CUDA when model is half / mixed-precision
    with torch.no_grad():
        # If you trained with fp16, you can also do:
        # with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device=="cuda")):
        generated_ids = model.generate(
            input_features=input_features,     # <-- KEY FIX: use keyword input_features
            max_new_tokens=max_new_tokens
        )

    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return transcription

for i in range(min(200, len(test_dataset))):
    print(f'i={i}')
    sid = test_dataset.metadata[i]["signal"]
    audio_path = TEST_AUDIO_DIR / f"{sid}.flac"
    pred_text = transcribe_sample(audio_path)
    ref_text  = test_dataset.metadata[i].get("prompt", "")
    loss_level  = test_dataset.metadata[i].get("hearing_loss", "")
    print(f"🎵 {sid}")
    print(f"Loss Level: {loss_level}")
    print(f"Prediction: {pred_text}")
    print(f"Reference : {ref_text}\n")


ValueError: module functions cannot set METH_CLASS or METH_STATIC

In [ ]:
!ls -R /content/drive/MyDrive/project/whisper_lora_results | head -n 40



In [15]:
import json

path = "/content/drive/MyDrive/project/whisper_lora_results/checkpoint-6603/trainer_state.json"
with open(path) as f:
    state = json.load(f)

for log in state["log_history"][-10:]:
    print(log)



{'epoch': 2.995910949568378, 'grad_norm': 5.60123872756958, 'learning_rate': 1.5260186174271326e-06, 'loss': 0.1223, 'step': 6594}
{'epoch': 2.996365288505225, 'grad_norm': 3.2007691860198975, 'learning_rate': 1.3734167556844193e-06, 'loss': 0.0939, 'step': 6595}
{'epoch': 2.996819627442072, 'grad_norm': 4.506551742553711, 'learning_rate': 1.220814893941706e-06, 'loss': 0.2021, 'step': 6596}
{'epoch': 2.9972739663789185, 'grad_norm': 1.4822306632995605, 'learning_rate': 1.0682130321989929e-06, 'loss': 0.052, 'step': 6597}
{'epoch': 2.9977283053157655, 'grad_norm': 4.027273178100586, 'learning_rate': 9.156111704562796e-07, 'loss': 0.289, 'step': 6598}
{'epoch': 2.9981826442526125, 'grad_norm': 3.4031805992126465, 'learning_rate': 7.630093087135663e-07, 'loss': 0.0243, 'step': 6599}
{'epoch': 2.9986369831894595, 'grad_norm': 4.364160537719727, 'learning_rate': 6.10407446970853e-07, 'loss': 0.205, 'step': 6600}
{'epoch': 2.999091322126306, 'grad_norm': 1.0384758710861206, 'learning_rate':

In [16]:
!nvidia-smi



Fri Nov 14 10:23:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [17]:
import time
import evaluate

# Load WER metric
wer_metric = evaluate.load("wer")
print("✅ Loaded WER metric successfully!")

predictions, references = [], []

for i in range(len(test_dataset)):
    entry = test_dataset.metadata[i]
    audio_path = TEST_AUDIO_DIR / f"{entry['signal']}.flac"

    start = time.time()
    print(f"\n🟢 [{i+1}/{len(test_dataset)}] Processing {audio_path.name} ...", flush=True)

    pred = transcribe_sample(audio_path)
    duration = time.time() - start
    print(f"✅ Done in {duration:.2f} sec | Pred: {pred[:50]}...", flush=True)

    predictions.append(pred)
    references.append(entry.get("prompt", ""))

wer = wer_metric.compute(predictions=predictions, references=references)
print(f"\n WER on valid set: {wer * 100:.2f}%")



ValueError: module functions cannot set METH_CLASS or METH_STATIC

In [2]:
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-distutils -y


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [3]:
!pip install torch torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121


In [12]:
import os

for root, dirs, files in os.walk("/content/drive/MyDrive", topdown=True):
    print(root, dirs, files)
    break


/content/drive/MyDrive ['valid', 'metadata', 'train', 'manifest', 'Colab Notebooks', 'project', 'cadenza_2026_submission'] ['lora fine tune4.ipynb', 'lora fine tune3.ipynb', 'whisper_baseline_results.csv']


In [13]:
!ls -R /content/drive/MyDrive/metadata


/content/drive/MyDrive/metadata:
prompts_phonemes.json  train_metadata_with_duration.json  valid_metadata.json
train_metadata.json    train_metadata_with_sa_stoi.json


In [21]:
# ============================
# 1. Imports
# ============================
import json
import numpy as np
import pandas as pd
import torch
import torchaudio
from jiwer import wer
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel
from pathlib import Path
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


# ============================
# 2. Load metadata
# ============================
metadata_path = "/content/drive/MyDrive/metadata/valid_metadata.json"

with open(metadata_path, "r") as f:
    metadata = json.load(f)

print("Loaded", len(metadata), "validation samples.")

# Build val_list
val_list = []
for item in metadata:
    sid = item["signal"]
    audio_path = f"/content/drive/MyDrive/valid/signals/{sid}.flac"

    gt_text = item.get("prompt") or item.get("original_prompt", "")
    gt_text = gt_text.strip().lower()

    val_list.append({
        "audio_path": audio_path,
        "text": gt_text,
        "signal": sid,
        "hearing_loss": item.get("hearing_loss", "NoLoss")
    })


# ============================
# 3. Load Whisper + LoRA
# ============================
BASE_MODEL = "openai/whisper-small"
LORA_MODEL_DIR = "/content/drive/MyDrive/project/whisper_lora_finetuned"

processor = WhisperProcessor.from_pretrained(BASE_MODEL)
base_model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL)
model = PeftModel.from_pretrained(base_model, LORA_MODEL_DIR)

model.to(device)
model.eval()

print("Whisper + LoRA loaded.\n")


# ============================
# Safe audio loader
# ============================
def load_audio_safe(path, target_sr=16000):
    audio, sr = torchaudio.load(path)

    # Fix multi-channel
    if audio.ndim == 2 and audio.shape[0] > 1:
        audio = audio.mean(dim=0, keepdim=True)

    # Fix corrupted shapes
    if audio.ndim != 2 or audio.shape[0] != 1:
        audio = audio.flatten().unsqueeze(0)

    # Resample if needed
    if sr != target_sr:
        audio = torchaudio.functional.resample(audio, sr, target_sr)

    return audio.squeeze()


# ============================
# 4. Inference + print + WER
# ============================
results = []

for item in tqdm(val_list):

    audio_path = item["audio_path"]
    gt = item["text"]

    # Load audio safely
    audio = load_audio_safe(audio_path)

    # Whisper inference
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt").to(device)

    with torch.no_grad():
        pred_ids = model.generate(
            input_features=inputs["input_features"],
            max_length=448
        )

    pred_text = processor.batch_decode(pred_ids, skip_special_tokens=True)[0]
    pred_text = pred_text.strip().lower()

    # Compute WER
    error = wer(gt, pred_text)

    # ====== REALTIME PRINT OUTPUT =========
    print("\n----------------------------------------")
    print(f"Signal:       {item['signal']}")
    print(f"Hearing Loss: {item['hearing_loss']}")
    print(f"Prediction:   {pred_text}")
    print(f"Ground Truth: {gt}")
    print(f"WER:          {error:.3f}")
    print("----------------------------------------\n")

    # Save result
    results.append({
        "signal": item["signal"],
        "audio_path": audio_path,
        "ground_truth": gt,
        "prediction": pred_text,
        "wer": error,
        "hearing_loss": item["hearing_loss"]
    })


# ============================
# 5. Print summary WER
# ============================
all_wers = [r["wer"] for r in results]

print("\n=====================================")
print(f"Mean WER: {np.mean(all_wers) * 100:.2f}%")
print("=====================================\n")


# ============================
# 6. Save CSV
# ============================
df = pd.DataFrame(results)
csv_path = "/content/drive/MyDrive/project/whisper_lora_predictions_with_wer.csv"
df.to_csv(csv_path, index=False)

print("Saved detailed prediction CSV to:", csv_path)


Device: cuda
Loaded 1175 validation samples.
Whisper + LoRA loaded.



  0%|          | 1/1175 [00:00<10:37,  1.84it/s]


----------------------------------------
Signal:       c657dc0bd1f822f8ff7104a1
Hearing Loss: Moderate
Prediction:   true love is known i am true to you more
Ground Truth: do you know i am truly alone
WER:          1.143
----------------------------------------



  0%|          | 2/1175 [00:01<09:37,  2.03it/s]


----------------------------------------
Signal:       5ab5a95be3c18dc173fdc18e
Hearing Loss: Moderate
Prediction:   down a hands in hands we are on the floor
Ground Truth: do not ask me that we are partners
WER:          1.000
----------------------------------------



...


----------------------------------------
Signal:       ec5ca82654a252fd1c673531
Hearing Loss: Mild
Prediction:   can you hear me and do you say those words
Ground Truth: can you hear me and do not you say those words
WER:          0.091
----------------------------------------



100%|█████████▉| 1174/1175 [20:14<00:01,  1.01s/it]


----------------------------------------
Signal:       6fd06b828c785b77e19dfdcb
Hearing Loss: No Loss
Prediction:   hate it as shiver and six t
Ground Truth: head down all shivering as if to yell
WER:          1.000
----------------------------------------



100%|██████████| 1175/1175 [20:14<00:00,  1.03s/it]


----------------------------------------
Signal:       5965000d1139d388ebcded80
Hearing Loss: Mild
Prediction:   we are had time to spend together
Ground Truth: we had time to spend together
WER:          0.167
----------------------------------------


Mean WER: 69.00%

Saved detailed prediction CSV to: /content/drive/MyDrive/project/whisper_lora_predictions_with_wer.csv


In [16]:
import json

with open("/content/drive/MyDrive/metadata/valid_metadata.json", "r") as f:
    data = json.load(f)

print("Example entry:\n")
print(data[0])


Example entry:

{'signal': 'c657dc0bd1f822f8ff7104a1', 'original_prompt': 'do you know i am truly alone', 'prompt': 'do you know i am truly alone', 'n_words': 7, 'hearing_loss': 'Moderate'}
